# Montana Climate and Water Data Demo

This notebook demonstrates how to access Montana-specific data sources:

1. **Montana Mesonet** (Montana Climate Office) - Weather and soil moisture stations
2. **GWIC** (Ground Water Information Center) - Well logs and groundwater data  
3. **DNRC StAGE** - Stream gage data
4. **Montana State Library** - GIS datasets

## Resources
- Montana Climate Office: https://climate.umt.edu/
- GWIC: https://mbmggwic.mtech.edu/
- DNRC Water Resources: https://dnrc.mt.gov/Water-Resources/
- MT State Library GIS: https://msl.mt.gov/geoinfo/

In [ ]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path
from datetime import datetime, timedelta

sys.path.insert(0, str(Path.cwd().parent.parent))

from scripts.data_retrieval import (
    MontanaMesonet,
    MontanaGWIC, 
    MontanaDNRC,
    MontanaStateLibrary,
    get_montana_clients,
)
from scripts.data_retrieval import (
    generate_mesonet_stations,
    generate_mesonet_observations,
)
from scripts.visualization import point_map, time_series_plot, histogram, set_theme

# Configuration: Set to True to use sample data instead of API calls
OFFLINE_MODE = False

# Set visualization theme
set_theme('light')

## 1. Montana Mesonet (Climate Office)

The Montana Mesonet is a network of 200+ weather and soil moisture monitoring stations.
Data is freely available through their REST API with no authentication required.

**API Documentation:** https://climate.umt.edu/mesonet/api/

In [ ]:
# Initialize the client
mesonet = MontanaMesonet()

# See available data elements
print("Available Mesonet data elements:")
for key, desc in mesonet.ELEMENTS.items():
    print(f"  {key}: {desc}")

In [ ]:
if OFFLINE_MODE:
    # Use synthetic sample data
    stations = generate_mesonet_stations(n_stations=50)
    print(f"Generated {len(stations)} sample Mesonet stations")
else:
    # Get all active stations
    try:
        stations = mesonet.get_stations(active_only=True)
        print(f"Found {len(stations)} active Mesonet stations")
    except Exception as e:
        print(f"API request failed: {e}")
        print("Falling back to sample data...")
        stations = generate_mesonet_stations(n_stations=50)
        print(f"Generated {len(stations)} sample stations")

stations.head()

In [ ]:
# Get stations as a GeoDataFrame for mapping
stations_gdf = mesonet.get_stations(as_geodataframe=True)
print(f"GeoDataFrame CRS: {stations_gdf.crs}")
stations_gdf.head()

In [ ]:
# Map all Mesonet stations
m = point_map(
    stations,
    lat_col='latitude',
    lon_col='longitude',
    popup_cols=['station', 'name', 'county', 'elevation'],
    center=[47.0, -110.0],  # Montana center
    zoom=6,
    tiles='CartoDB positron',
)
m

In [ ]:
# Get latest observations from all stations
latest = mesonet.get_latest()
print(f"Latest data from {len(latest)} stations")
latest.head()

In [ ]:
# Find stations in a specific county
gallatin_stations = mesonet.search_stations_by_county("Gallatin")
print(f"Found {len(gallatin_stations)} stations in Gallatin County:")
gallatin_stations[['station', 'name', 'latitude', 'longitude']]

In [ ]:
# Get hourly data for a station
if OFFLINE_MODE:
    # Generate sample observations
    hourly = generate_mesonet_observations(
        stations.head(5),  # Use first 5 stations
        start_date=datetime.now() - timedelta(days=7),
        frequency='hourly',
    )
    station_name = "Sample Station"
    print(f"Generated {len(hourly)} sample hourly observations:")
    hourly.head(10)
elif not gallatin_stations.empty:
    station_id = gallatin_stations.iloc[0]['station']
    station_name = gallatin_stations.iloc[0]['name']
    
    try:
        hourly = mesonet.get_hourly_observations(
            stations=[station_id],
            start_date=datetime.now() - timedelta(days=7),
            elements=['air_temp', 'relative_humidity', 'ppt'],
        )
        print(f"Hourly data for {station_name} ({station_id}):")
        hourly.head(10)
    except Exception as e:
        print(f"API request failed: {e}")
        hourly = generate_mesonet_observations(
            stations.head(5),
            start_date=datetime.now() - timedelta(days=7),
            frequency='hourly',
        )
        station_name = "Sample Station"
        print(f"Generated {len(hourly)} sample observations instead")
else:
    hourly = pd.DataFrame()
    station_name = "No station"
    print("No Gallatin County stations found")

In [ ]:
# Plot temperature time series
if 'hourly' in dir() and not hourly.empty:
    fig = time_series_plot(
        hourly,
        date_col='datetime',
        value_col='air_temp',
        title=f'Air Temperature at {station_name}',
        ylabel='Temperature (°C)',
    )

In [ ]:
# Get daily observations for longer period
if not gallatin_stations.empty:
    daily = mesonet.get_daily_observations(
        stations=[station_id],
        start_date=datetime.now() - timedelta(days=90),
        elements=['air_temp', 'ppt'],
    )
    
    print(f"Daily data ({len(daily)} days):")
    daily.head()

In [ ]:
# Get derived agricultural metrics (ETo, Growing Degree Days)
if not gallatin_stations.empty:
    derived = mesonet.get_derived_metrics(
        stations=[station_id],
        start_date=datetime.now() - timedelta(days=30),
    )
    
    if not derived.empty:
        print("Derived metrics:")
        display(derived.head())
    else:
        print("No derived metrics available for this station")

## 2. GWIC (Ground Water Information Center)

GWIC is Montana's groundwater database maintained by the Montana Bureau of Mines and Geology.
It contains well logs, water levels, and water quality data for wells across the state.

**Website:** https://mbmggwic.mtech.edu/

In [ ]:
gwic = MontanaGWIC()

# Get statewide monitoring network wells
# These are long-term sites with historical water level data
monitoring_wells = gwic.get_monitoring_network_wells()

if not monitoring_wells.empty:
    print(f"Found {len(monitoring_wells)} statewide monitoring wells")
    display(monitoring_wells.head())
else:
    print("Could not retrieve monitoring wells via ArcGIS service.")
    print(f"Visit GWIC directly: https://mbmggwic.mtech.edu/")

In [ ]:
# Query wells in a specific area (bounding box)
# Example: Bozeman area
bozeman_bbox = (-111.2, 45.6, -110.8, 45.8)  # (xmin, ymin, xmax, ymax)

bozeman_wells = gwic.get_wells_from_arcgis(bbox=bozeman_bbox)

if not bozeman_wells.empty:
    print(f"Found {len(bozeman_wells)} wells in Bozeman area")
    display(bozeman_wells.head())
else:
    print("No wells returned from ArcGIS query.")
    print("Note: The GWIC ArcGIS services may have access restrictions.")

In [ ]:
# Get the GWIC URL for a specific well
example_gwic_id = "123456"  # Replace with actual GWIC ID
print(f"GWIC well page: {MontanaGWIC.get_gwic_url(example_gwic_id)}")

## 3. DNRC StAGE (Stream and Gage Explorer)

DNRC operates stream gages across Montana for water management.
The StAGE web application provides real-time and historical streamflow data.

**Website:** https://gis.dnrc.mt.gov/apps/stage/

In [ ]:
dnrc = MontanaDNRC()

# Get stream gages
gages = dnrc.get_stream_gages()

if not gages.empty:
    print(f"Found {len(gages)} DNRC stream gages")
    display(gages.head())
else:
    print("Could not retrieve stream gages via ArcGIS service.")
    print(f"Visit StAGE directly: https://gis.dnrc.mt.gov/apps/stage/")

In [ ]:
# Map stream gages if available
if not gages.empty and 'latitude' in gages.columns:
    m = point_map(
        gages.dropna(subset=['latitude', 'longitude']),
        lat_col='latitude',
        lon_col='longitude',
        center=[47.0, -110.0],
        zoom=6,
        cluster=True,
    )
    display(m)

In [ ]:
# Get water rights places of use (for a county)
water_rights = dnrc.get_water_rights_pou(county="Gallatin")

if not water_rights.empty:
    print(f"Found {len(water_rights)} water rights POUs in Gallatin County")
    display(water_rights.head())
else:
    print("Could not retrieve water rights data.")
    print(f"Visit WRQS: {MontanaDNRC.get_wrqs_url()}")

In [ ]:
# Get StAGE URL for a specific gage
example_site_id = "GAGE001"  # Replace with actual site ID
print(f"StAGE gage page: {MontanaDNRC.get_stage_url(example_site_id)}")

## 4. Montana State Library GIS Data

The Montana State Library provides statewide GIS datasets through MSDI.

**Data Catalog:** https://mslservices.mt.gov/geographic_information/data/datalist/

In [ ]:
msl = MontanaStateLibrary()

print("Montana State Library Resources:")
print(f"  Data Catalog: {msl.get_data_catalog_url()}")
print()
print("Available dataset categories:")
for key, path in msl.DATASETS.items():
    print(f"  {key}: {path}")

## Combining Federal and State Data

You can combine Montana state data with federal sources (USGS, NOAA) for comprehensive analysis.

In [ ]:
from scripts.data_retrieval import USGSWaterServices

# Get USGS sites in Montana
usgs = USGSWaterServices()

# Groundwater sites in Montana
usgs_gw_sites = usgs.get_sites(
    state_code="MT",
    site_type="GW",
)
print(f"USGS groundwater sites in Montana: {len(usgs_gw_sites)}")

# Stream sites in Montana
usgs_stream_sites = usgs.get_sites(
    state_code="MT",
    site_type="ST",
)
print(f"USGS stream sites in Montana: {len(usgs_stream_sites)}")

In [ ]:
# Example: Compare Mesonet stations with USGS sites in same area
print("\nData source comparison for Montana:")
print(f"  Montana Mesonet stations: {len(stations)}")
if not monitoring_wells.empty:
    print(f"  GWIC monitoring wells: {len(monitoring_wells)}")
if not gages.empty:
    print(f"  DNRC stream gages: {len(gages)}")
print(f"  USGS groundwater sites: {len(usgs_gw_sites)}")
print(f"  USGS stream sites: {len(usgs_stream_sites)}")

## Data Source Quick Reference

| Source | Data Types | API Available | Notes |
|--------|------------|---------------|-------|
| Montana Mesonet | Weather, soil moisture | Yes (REST) | No auth required, 5-min updates |
| GWIC | Well logs, water levels, water quality | ArcGIS Services | Web interface most complete |
| DNRC StAGE | Stream flow, temperature | ArcGIS Services | Real-time data available |
| MT State Library | GIS datasets | FTP/Downloads | Statewide boundaries, hydrography |
| USGS (federal) | All water data types | Yes (REST) | Extensive Montana coverage |

In [ ]:
# Get all Montana clients at once
clients = get_montana_clients()
print("Available Montana data clients:")
for name, client in clients.items():
    print(f"  {name}: {type(client).__name__}")